In [3]:
import os
import re
import json
import shutil
import time
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

# Charger les variables d'environnement
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("La clé API OpenAI n'a pas été trouvée dans le fichier .env")

feedback_dir = "data/feedback"
processed_dir = "data/feedback/feedback_processed"
jsonl_path = "data/decision_examples.jsonl"
os.makedirs(processed_dir, exist_ok=True)

def parse_feedback_file(filepath: str):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    section_regex = r"-\s*Section actuelle\s*:\s*(\d+)"
    user_response_regex = r"-\s*Réponse utilisateur\s*:\s*(.*)"
    decision_regex = r"-\s*Décision prise\s*:\s*Section\s+(\d+)"
    section_content_regex = r"## Section\s*(.*?)\s*## Feedback"
    feedback_regex = r"## Feedback\s*(.*)"

    section_match = re.search(section_regex, content)
    user_response_match = re.search(user_response_regex, content)
    decision_match = re.search(decision_regex, content)
    section_content_match = re.search(section_content_regex, content, re.DOTALL)
    feedback_match = re.search(feedback_regex, content, re.DOTALL)

    if not (section_match and user_response_match and decision_match and section_content_match and feedback_match):
        print(f"Impossible de parser le feedback dans {filepath}")
        return None

    section = int(section_match.group(1))
    user_response = user_response_match.group(1).strip()
    taken_decision = int(decision_match.group(1))
    section_text = section_content_match.group(1).strip()
    feedback_text = feedback_match.group(1).strip()

    return {
        "filepath": filepath,
        "section": section,
        "user_response": user_response,
        "taken_decision": taken_decision,
        "section_text": section_text,
        "feedback_text": feedback_text
    }

def analyze_error_with_langchain(section: int, user_response: str, taken_decision: int, feedback_text: str, section_text: str) -> str:
    prompt_template = PromptTemplate(
        input_variables=["section", "user_response", "taken_decision", "feedback_text", "section_text"],
        template="""
Voici un retour utilisateur sur une décision prise par l'IA dans une histoire interactive.

- Section actuelle : {section}
- Réponse de l'utilisateur (choix souhaité) : {user_response}
- Décision prise par l'IA : {taken_decision}

Contenu de la section actuelle :
{section_text}

Feedback de l'utilisateur sur cette erreur :
{feedback_text}

Analyse de façon terre à terre et concise :
1. Indique quelle décision l'IA aurait dû prendre logiquement.
2. Dis si le feedback est justifié en fonction de la logique du jeu.
N'évoque aucune émotion, reste purement logique.
"""
    )

    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=openai_api_key,
        model_name="gpt-4o-mini"
    )

    chain = LLMChain(llm=llm, prompt=prompt_template)
    analysis = chain.run(
        section=section,
        user_response=user_response,
        taken_decision=taken_decision,
        feedback_text=feedback_text,
        section_text=section_text
    )
    return analysis.strip()

def append_feedback_to_jsonl(section: int, user_response: str, taken_decision: int, analysis: str, feedback_text: str, section_text: str):
    system_content = "l'agent de décision doit comprendre les décisions de l'utilisateur en fonction du choix donné par le narrateur"
    user_content = (
        f"Section: {section}, l'utilisateur voulait '{user_response}', "
        f"l'IA a décidé section {taken_decision}. Section actuelle:\n{section_text}\n"
        f"Feedback: {feedback_text}"
    )
    assistant_content = analysis

    # On ne met pas de type, ni de feedback_section dans le JSONL, seulement messages.
    entry = {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ]
    }

    print("Nouvelle entrée ajoutée au JSONL :")
    print(json.dumps(entry, ensure_ascii=False, indent=2))

    # Écriture dans le JSONL
    with open(jsonl_path, 'a', encoding='utf-8') as f:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    # Print la section dans la console sans l'ajouter au JSONL
    print(f"Section {section} traitée (non incluse dans le JSONL, juste affichée).")

def process_all_feedback():
    processed_sections = []
    files = [f for f in os.listdir(feedback_dir) if f.endswith(".md")]
    if not files:
        print("Aucun fichier de feedback à traiter.")
        return processed_sections

    for filename in files:
        filepath = os.path.join(feedback_dir, filename)
        fb = parse_feedback_file(filepath)

        if fb is None:
            print(f"Feedback non parsé pour {filename}, fichier laissé en place.")
            continue

        analysis = analyze_error_with_langchain(
            fb["section"],
            fb["user_response"],
            fb["taken_decision"],
            fb["feedback_text"],
            fb["section_text"]
        )

        append_feedback_to_jsonl(
            fb["section"],
            fb["user_response"],
            fb["taken_decision"],
            analysis,
            fb["feedback_text"],
            fb["section_text"]
        )

        shutil.move(filepath, os.path.join(processed_dir, filename))
        print(f"Feedback {filename} traité, fichier déplacé.")
        processed_sections.append(fb["section"])

    return processed_sections

# Exécuter le traitement de tous les feedbacks
processed_sections = process_all_feedback()
print("Sections traitées lors de cette exécution:", processed_sections)


Nouvelle entrée ajoutée au JSONL :
{
  "messages": [
    {
      "role": "system",
      "content": "l'agent de décision doit comprendre les décisions de l'utilisateur en fonction du choix donné par le narrateur"
    },
    {
      "role": "user",
      "content": "Section: 246, l'utilisateur voulait 'je ne suis pas chanceux', l'IA a décidé section 246. Section actuelle:\n# Section 246\n\n— Votre Excellence, bredouillez-vous d'une pauvre voix, je ne pouvais supposer... je veux dire qu'il ne m'est pas venu à l'idée que ce vaisseau pouvait être piloté par l'un des glorieux maîtres du noble Empire arcadien. Mon unique désir est de servir Arcadion. Je croyais avoir affaire à un vaisseau pirate et mon seul but a été de protéger la précieuse cargaison que je transportais. Tentez votre Chance pour savoir si l'Arcadien se laissera abusé par votre histoire. Si vous êtes Chanceux, rendez-vous au [[144]]. Sinon, rendez-vous au [[129]].\nFeedback: je ne suis pas chanceux, donc je dois aller au 129

In [4]:
import os
import re
import json
import time
import faiss
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate

# Charger les variables d'environnement
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("La clé API OpenAI n'a pas été trouvée dans le fichier .env")

os.environ["OPENAI_API_KEY"] = openai_api_key

jsonl_path = 'data/decision_examples.jsonl'
sections_dir = 'data/sections'
output_directory = 'data/rules'

# Dossier dédié à l'index decision_index
index_dir = 'data/index/decision_index'
faiss_index_path = os.path.join(index_dir, 'decision_index.faiss')
docstore_path = os.path.join(index_dir, 'docstore.json')

if not os.path.exists(jsonl_path):
    raise FileNotFoundError(f"Le fichier {jsonl_path} n'a pas été trouvé.")

jsonl_mtime = os.path.getmtime(jsonl_path)
index_exists = os.path.exists(faiss_index_path) and os.path.exists(docstore_path)

recreate_index = False
if index_exists:
    faiss_mtime = os.path.getmtime(faiss_index_path)
    if jsonl_mtime > faiss_mtime:
        print("Le fichier JSONL a été modifié après la dernière mise à jour de l'index FAISS.")
        recreate_index = True
    else:
        print("L'index FAISS est à jour.")
else:
    print("L'index FAISS n'existe pas et sera créé.")
    recreate_index = True

def create_or_update_index():
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                obj = json.loads(line)
                data.append(obj)
    
    texts = []
    for entry in data:
        if 'text' in entry:
            texts.append(entry['text'])
        elif 'messages' in entry and isinstance(entry['messages'], list):
            combined = "\n".join(msg["content"] for msg in entry["messages"] if "content" in msg)
            if combined.strip():
                texts.append(combined)

    if not texts:
        print("Aucun texte ni messages pour construire l'index.")
        return None, None, None

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )
    
    all_chunks = []
    for t in texts:
        all_chunks.extend(text_splitter.split_text(t))
    
    embeddings = OpenAIEmbeddings()
    embedding_vectors = embeddings.embed_documents(all_chunks)
    if not embedding_vectors:
        print("Aucune embedding générée.")
        return None, None, None

    embedding_dim = len(embedding_vectors[0])
    
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(np.array(embedding_vectors).astype('float32'))
    
    docstore = {str(i): chunk for i, chunk in enumerate(all_chunks)}
    
    os.makedirs(index_dir, exist_ok=True)
    faiss.write_index(index, faiss_index_path)
    print(f"Index FAISS sauvegardé dans {faiss_index_path}.")
    
    with open(docstore_path, 'w', encoding='utf-8') as f:
        json.dump(docstore, f)
    print(f"Docstore sauvegardé dans {docstore_path}.")
    
    return index, docstore, embeddings

if recreate_index:
    index, docstore, embeddings = create_or_update_index()
    if index is None:
        raise SystemExit("Impossible de créer l'index RAG (pas de texte).")
else:
    if not (os.path.exists(faiss_index_path) and os.path.exists(docstore_path)):
        print("Index ou docstore manquant, recréez l'index.")
        raise SystemExit
    index = faiss.read_index(faiss_index_path)
    with open(docstore_path, 'r', encoding='utf-8') as f:
        docstore = json.load(f)
    embeddings = OpenAIEmbeddings()
    print("Index FAISS et docstore chargés avec succès.")

def process_section(section_number: int):
    markdown_path = os.path.join(sections_dir, f"{section_number}.md")
    if not os.path.exists(markdown_path):
        print(f"La section {section_number} n'existe pas.")
        return

    with open(markdown_path, 'r', encoding='utf-8') as f:
        section_content = f.read()

    query = ("Donne des exemples pour distinguer l'histoire du livre dont vous êtes le héros "
             "des règles, choix et décisions purement ludiques. Aide-moi à comprendre, mais "
             "je n'intégrerai pas ces exemples dans le résultat final.")
    query_embedding = embeddings.embed_query(query)
    query_vector = np.array([query_embedding]).astype('float32')

    k = 5
    distances, indices = index.search(query_vector, k)

    retrieved_texts = [docstore[str(i)] for i in indices[0] if str(i) in docstore]
    rag_instructions = "\n\n".join(retrieved_texts)

    prompt_template = """
Tu es un assistant chargé d'extraire les règles du jeu présentées dans une section d'un livre dont vous êtes le héros.
Le texte RAG ci-dessous fournit des exemples pour t'aider à comprendre comment séparer l'histoire des vraies règles du jeu.
Ne reprends pas ces exemples dans le résultat final, ils sont juste pour t'aider à comprendre.

RAG (exemples, ne pas les intégrer dans le résultat) :
{rag_instructions}

Section :
{section_content}

Instructions :
- Distingue l'histoire (description, narration) des éléments ludiques (choix du joueur, gain/perte de stats, argent, chemins à prendre, combat, énigme...).
- N'inclus que les règles et décisions spécifiques à cette section.
- Formate les règles en Markdown, par exemple :
  - Choix 1: ...
  - Endurance, CHance, Habileté +x si ...
  - Gagner, perdre de l'argent si
  - choisir cette action mène à ...
  - devoir se battre contre un monstre si
  - si je gagne une partie, je ...
  
Extrais uniquement ces éléments ludiques propres à cette section, sans reprendre les exemples du RAG.
"""
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["rag_instructions", "section_content"]
    )

    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=openai_api_key,
        model_name="gpt-4o-mini"
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    rules = chain.run(rag_instructions=rag_instructions, section_content=section_content)

    os.makedirs(output_directory, exist_ok=True)
    output_filename = f'section_{section_number}_rule.md'
    output_path = os.path.join(output_directory, output_filename)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(rules)

    print(f"Les règles de la section {section_number} ont été enregistrées dans {output_path}")

#############################
# Modes de traitement
#############################

# Supposons que nous ayons trois modes :
# 1. "new" : traite uniquement les sections dans processed_sections (viennent d'être traitées par le script précédent)
# 2. "list" : traite une liste de sections choisies (par exemple [1,48,164])
# 3. "all" : traite toutes les sections disponibles

mode = "new"  # Changez le mode selon le besoin

# Exemple : récupéré depuis un script précédent
processed_sections = [48, 164]  # Sections provenant du script précédent

# Liste d'exemples, si mode = "list"
chosen_sections = [1, 48, 164]

if mode == "new":
    sections_to_process = processed_sections
elif mode == "list":
    sections_to_process = chosen_sections
elif mode == "all":
    all_files = os.listdir(sections_dir)
    md_files = [f for f in all_files if f.endswith(".md")]
    all_sections = []
    for mf in md_files:
        m = re.search(r'(\d+)\.md', mf)
        if m:
            sec_num = int(m.group(1))
            all_sections.append(sec_num)
    sections_to_process = sorted(all_sections)
else:
    sections_to_process = []

if isinstance(sections_to_process, int):
    sections_to_process = [sections_to_process]

# Multi-threading pour traiter les sections
num_threads = min(len(sections_to_process), 4)

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = {executor.submit(process_section, sec): sec for sec in sections_to_process}
    for future in as_completed(futures):
        sec = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Erreur lors du traitement de la section {sec}: {e}")


Le fichier JSONL a été modifié après la dernière mise à jour de l'index FAISS.
Index FAISS sauvegardé dans data/index/decision_index\decision_index.faiss.
Docstore sauvegardé dans data/index/decision_index\docstore.json.
Les règles de la section 164 ont été enregistrées dans data/rules\section_164_rule.md
Les règles de la section 48 ont été enregistrées dans data/rules\section_48_rule.md
